In [ ]:
import os
import pandas as pd
import numpy
from datetime import timedelta
import numpy as np
import sys

os.chdir('/opt/jhub_shared/')
sys.path.append(os.getcwd())

from common.prediction.data_engineering.data_preparation import resample_and_time_adj

In [5]:
# Define the list of sustainability metrics we'd wish to calculate 
# 1. % acceptance/rejection
# 2. top reasons for rejection
# 3. % of times within 25/50/75 % of the recommended value

In [2]:
# Read in the feedback table and the metadata table
# metadata = pd.read_csv('/opt/jhub_shared/DryerOpt_project/outputs/recs/rec_review/metadata_ga_and_hill_climb_pfa_2018_06_03_00__00__00.csv')
# feedback = pd.read_csv('/opt/jhub_shared/DryerOpt_project/outputs/recs/rec_review/recommendation_ga_and_hill_climb_pfa_2018_06_03_00__00__00.csv')
metadata = pd.read_csv('/opt/jhub_shared/pfa/data/processed/metadata_table.csv')
feedback = pd.read_csv('/opt/jhub_shared/pfa/data/processed/feedback_table.csv')

# Also read in the raw data from IP21 to calculate some of the metrics
df_raw = pd.read_pickle('/opt/jhub_shared/common/outputs/intermediate/dataframes/data_processed_p1630tg.pk')

In [24]:
# Grab the recommended features
rec_features = list(set(list(feedback['feature'])) - set(['Dryer exit temperature']))

In [3]:
for exec_id in metadata['ExecutionID']:
    

,ExecutionID,ExecuteDate,test_shift,test_algorithm,pfa_test_model,filename,ExecutionTime,Rate change,Temperature change
0,20200000000001,39:23.5,6/3/2018 0:00,ga_and_hill_climb,MLP,ga_and_hill_climb_pfa_2018_06_03_00__00__00.csv,527.859702,10.34,-0.48
1,20200000000002,29:46.9,6/5/2018 9:00,ga_and_hill_climb,MLP,ga_and_hill_climb_pfa_2018_06_05_09__00__00.csv,528.794836,10.34,1.47
2,20200000000003,49:01.0,10/8/2018 0:00,ga_and_hill_climb,MLP,ga_and_hill_climb_pfa_2018_10_08_00__00__00.csv,528.897241,10.17,0.23
3,20200000000004,38:49.8,11/11/2018 15:00,ga_and_hill_climb,MLP,ga_and_hill_climb_pfa_2018_11_11_15__00__00.csv,153.492484,10.91,0.26
4,20200000000005,42:19.6,11/17/2018 12:00,ga_and_hill_climb,MLP,ga_and_hill_climb_pfa_2018_11_17_12__00__00.csv,158.522596,11.54,-0.49


In [25]:
# Fetch the timestamp from the metadata, take the corresponding process data, and compare it to the actual recs
metadata_exec = metadata[metadata['ExecutionID'] == exec_id]

start_time = metadata_exec['test_shift'].values[0]
end_time = str(pd.to_datetime(start_time) + timedelta(hours = 3))

# Grab the process data corresponding with this metadata
process_data = df_raw.loc[(df_raw.index >= start_time) & (df_raw.index < end_time)][rec_features]
process_data['timestamp'] = process_data.index
process_data['timestamp'] = process_data['timestamp'].astype('object')
process_data['timestamp'] = [str(f)[:19] for f in process_data['timestamp']]

In [67]:
feedback['timestamp'] = start_time
recs_table = feedback.pivot(index='timestamp', columns = 'feature', values = 'Recommendations').drop(['Dryer exit temperature'], axis = 1)
recs_table.columns = [str(f) + '_rec' for f in recs_table.columns]
recs_table['timestamp'] = recs_table.index
recs_table.reset_index(drop = True, inplace = True)
recs_table = recs_table.merge(metadata[['ExecutionID', 'test_shift']], how = 'left', left_on = 'timestamp', right_on = 'test_shift')
recs_table.drop(['timestamp', 'test_shift'], axis = 1, inplace=True)

In [68]:
time_range = pd.date_range(start_time, end_time, freq='T', closed = 'left').to_list()
time_range_df =pd.DataFrame({})
time_range_df['timestamp'] = [str(f) for f in time_range]
time_range_df['ExecutionID'] = metadata['ExecutionID'].values[0]

In [69]:
# Each of these timestamps should have the corresponding values of the recs and the values of the actual process
time_range_df = time_range_df.merge(recs_table, how = 'left', on = 'ExecutionID')
time_range_df = time_range_df.merge(process_data, how = 'left', on = 'timestamp')

In [101]:
# Write a function and a loop to take pair of actual and rec values and generate the above metrics
def calculate_perc_metrics(feat, time_range_df):
    feat_rec = feat + '_rec'
    #temp = time_range_df[[feat, feat + '_rec']]
    perc_within_25 = sum(abs(time_range_df[feat_rec] - time_range_df[feat])/time_range_df[feat_rec] <= 0.25)/len(time_range_df)
    perc_within_50 = sum(abs(time_range_df[feat_rec] - time_range_df[feat])/time_range_df[feat_rec] <= 0.5)/len(time_range_df)
    perc_within_75 = sum(abs(time_range_df[feat_rec] - time_range_df[feat])/time_range_df[feat_rec] <= 0.75)/len(time_range_df)
    avg_perc_diff = np.mean(abs(time_range_df[feat_rec] - time_range_df[feat])/time_range_df[feat_rec])

    temp_stats = pd.DataFrame({'times within 25%': perc_within_25,
                              'times within 50%': perc_within_50, 'times within 75%': perc_within_75,
                              'avg difference between rec and actual': avg_perc_diff}, index = [feat])
    return temp_stats

stats_df = pd.DataFrame({})
for feat in list(rec_features):
    #feat = rec_features[0]
    temp_stats = calculate_perc_metrics(feat, time_range_df)
    stats_df = pd.concat([stats_df, temp_stats])

In [7]:
# Calculate the other metrics
df_rate_recipe = df_raw[['recipe_number', 'p1630tg', 'p1548fc_sp']]

,p1630tg,p1548fc_sp
date,,
2017-07-11 04:00:00+00:00,195.875,2800.0
2017-07-11 04:01:00+00:00,195.750,2800.0
2017-07-11 04:02:00+00:00,195.625,2800.0
2017-07-11 04:03:00+00:00,195.625,2800.0
2017-07-11 04:04:00+00:00,195.625,2800.0


In [8]:
df_raw.shape

(965008, 1496)